# Combine the datasets in a single dataframe

In [103]:
import os

import numpy as np
import pandas as pd
import networkx as nx
import pickle
from tqdm import tqdm
import geopandas as gpd

from scipy import stats

import re
import unidecode

import matplotlib.pyplot as plt
%matplotlib inline

## Merge with UNHCR dataset

In [79]:
with open(os.path.join("pickle", 'conflict.pickle'), 'rb') as data_source:
    conflict_df = pickle.load(data_source)

In [80]:
with open(os.path.join("pickle", 'refugee.pickle'), 'rb') as data_source:
    refugee_df = pickle.load(data_source)

The goal of this section is to create a dataframe that links the displacement information given by the UNHCR with the events and deaths given by the GED dataset. This will be grouped by year and will keep track of the number of events and of the event identifiers.

The columns will be:
 - `country_id`
 - `year` 
 - `number_of_events` the number of events in a country in a year
 - `events_id` an array containing the id of each event accounted for
 - `displacement_extern` the sum of `[refugee, asylum, stateless, others]`
 - `displacement_intern` the value of internally_displaced
 - `total_displacement` 
 - `deaths`

In [81]:
COUNTRY_HUMAN_COST_DF_COLUMNS = [
    "country_id",
    "year",
    "number_of_events",
    "events_id",
    "displacement_extern",
    "displacement_intern",
    "total_displacement",
    "deaths"
]

### Manage index and data types

We sort and modify the indexing of our dataframes for easier and more performant data wrangling

In [82]:
# Sort conflict dataframe so we can easily iter through it
sorted_conflict_df = conflict_df.sort_values(["year", "country_id"], axis=0)
sorted_conflict_df.head(2)

,id,year,type_of_violence,conflict_new_id,conflict_name,side_a_new_id,gwnoa,side_a,gwnob,side_b_new_id,...,country,country_id,region,date_start,date_end,deaths_a,deaths_b,deaths_civilians,deaths_unknown,best
107883,186087,1989,1,381,Haiti:Government,6,41.0,Government of Haiti,NaN,763,...,Haiti,41,Americas,1989,1989,0,4,0,0,4
107890,186122,1989,1,381,Haiti:Government,6,41.0,Government of Haiti,NaN,763,...,Haiti,41,Americas,1989,1989,0,0,0,1,1


In [83]:
refugee_df["year"] = pd.to_numeric(refugee_df["year"])
refugee_df["country_id"] = pd.to_numeric(refugee_df["country_id"])

In [84]:
# Index refugee_df for easier search
try:
    refugee_df = refugee_df.set_index(["year", "country_id"])
except: # if we run this two times
    pass
# Get the displacement_extern value
refugee_df["displacement_extern"] = refugee_df.apply(\
                                    lambda row: row.refugee + row.asylum + row.stateless + row.others, axis=1)
refugee_df.head(1)

,,refugee,asylum,internally_displaced,stateless,others,total,displacement_extern
year,country_id,,,,,,,
1989.0,490.0,100786.0,0.0,0.0,0.0,0.0,100786.0,100786.0


In [85]:
refugee_df = refugee_df.sort_index()
refugee_df.head(5)

refugee  asylum  internally_displaced  stateless  others  \
year   country_id                                                             
1989.0 40.0         7682.0     0.0                   0.0        0.0     0.0   
       41.0         3344.0     0.0                   0.0        0.0     0.0   
       55.0            5.0     0.0                   0.0        0.0     0.0   
       90.0        45413.0     0.0                   0.0        0.0     0.0   
       91.0            5.0     0.0                   0.0        0.0     0.0   

                     total  displacement_extern  
year   country_id                                
1989.0 40.0         7682.0               7682.0  
       41.0         3344.0               3344.0  
       55.0            5.0                  5.0  
       90.0        45413.0              45413.0  
       91.0            5.0                  5.0

In [86]:
def get_total_deaths_from_event_ids(conflict_df, event_ids):
    """This function will return the total deaths associated with a list of event id"""
    deaths = 0
    for event in event_ids:
        index = conflict_df.index[conflict_df.id == event].tolist()[0]
        deaths += conflict_df.loc[index, "best"]
    return deaths

In [87]:
def get_append_series(conflict_df, refugee_df, year, country_id, event_ids=[]):
    """This function will create a serie to be appended to the country_human_cost_df with the proper values
       at each column
    """
    try:
        displacement_extern = refugee_df.displacement_extern[year, country_id]
        displacement_intern = refugee_df.internally_displaced[year, country_id]
        total_displacement  = refugee_df.total[year, country_id]
    except KeyError: # No value for this key
        displacement_extern = 0
        displacement_intern = 0
        total_displacement  = 0
        
    deaths = get_total_deaths_from_event_ids(conflict_df, event_ids)
    
    data = [ country_id, year, len(event_ids), event_ids,
             displacement_extern, displacement_intern,
             total_displacement, deaths
           ] 
    series_to_append = pd.Series(data = data, index=COUNTRY_HUMAN_COST_DF_COLUMNS)
    return series_to_append

We create the `country_human_cost_df` and fill it country by country, with different values for each year

In [88]:
# Init variables
current_year = None
current_country_id = None
event_ids = []

country_human_cost_df = pd.DataFrame(columns=COUNTRY_HUMAN_COST_DF_COLUMNS) 
for row in tqdm(sorted_conflict_df.itertuples()):
    # Check if we are still in the same year and country
    if (current_year != row.year) or (current_country_id != row.country_id):
        # If we registered event, add a row to the country_human_cost_df
        if event_ids:
            series_to_append = get_append_series(conflict_df, refugee_df, current_year, current_country_id, event_ids)
            country_human_cost_df = country_human_cost_df.append(series_to_append, ignore_index=True)
        
        # Set the variables to the desired values
        event_ids = []
        current_year = row.year
        current_country_id = row.country_id
        
    # Add the event to the list
    event_ids.append(row.id)

135181it [00:47, 2871.87it/s]


In [89]:
country_human_cost_df["country_id"] = pd.to_numeric(country_human_cost_df["country_id"])
country_human_cost_df["year"] = pd.to_numeric(country_human_cost_df["year"])
country_human_cost_df.head(5)

,country_id,year,number_of_events,events_id,displacement_extern,displacement_intern,total_displacement,deaths
0,41,1989,4,"[186087, 186122, 186123, 186124]",0,0,0,48
1,51,1989,1,[107752],0,0,0,1
2,70,1989,2,"[182047, 182048]",0,0,0,3
3,90,1989,78,"[193728, 193732, 193764, 193765, 193766, 19376...",0,0,0,491
4,92,1989,58,"[120534, 120536, 120537, 120538, 120539, 12054...",0,0,0,4924


We change the index again after merging:

In [90]:
try:
    country_human_cost_df = country_human_cost_df.set_index(["year", "country_id"])
except:  # In case cell is re-run
    pass
country_human_cost_df = country_human_cost_df.sort_index()
country_human_cost_df.head(5)

number_of_events  \
year country_id                    
1989 41                        4   
     51                        1   
     70                        2   
     90                       78   
     92                       58   

                                                         events_id  \
year country_id                                                      
1989 41                           [186087, 186122, 186123, 186124]   
     51                                                   [107752]   
     70                                           [182047, 182048]   
     90          [193728, 193732, 193764, 193765, 193766, 19376...   
     92          [120534, 120536, 120537, 120538, 120539, 12054...   

                displacement_extern displacement_intern total_displacement  \
year country_id                                                              
1989 41                           0                   0                  0   
     51                           0                   0                  0   
     70                           0                   0                  0   
     90                           0                   0                  0   
     92                           0                   0                  0   

                deaths  
year country_id         
1989 41             48  
     51              1  
     70              3  
     90            491  
     92           4924

We also take into account migrants displacement for countries where there was no conflict

In [91]:
new_country_human_cost_df = country_human_cost_df
for row in tqdm(refugee_df.itertuples()):
    try:
        country_human_cost_df.loc[row.Index]
    except KeyError: # The row is not in the country_human_cost_df
        series_to_append = get_append_series(conflict_df, refugee_df, row.Index[0], row.Index[1]) 
        df_to_append = pd.DataFrame([series_to_append.transpose()])
        df_to_append = df_to_append.set_index(["year", "country_id"])
        new_country_human_cost_df = new_country_human_cost_df.append(df_to_append)
    except TypeError:
        continue


4749it [01:11, 66.58it/s]


In [92]:
country_human_cost_df = new_country_human_cost_df

In [93]:
country_human_cost_df = country_human_cost_df.sort_index()
country_human_cost_df.head(5)

number_of_events                         events_id  \
year   country_id                                                      
1989.0 40.0                      0                                []   
       41.0                      4  [186087, 186122, 186123, 186124]   
       41.0                      0                                []   
       51.0                      1                          [107752]   
       55.0                      0                                []   

                  displacement_extern displacement_intern total_displacement  \
year   country_id                                                              
1989.0 40.0                      7682                   0               7682   
       41.0                         0                   0                  0   
       41.0                      3344                   0               3344   
       51.0                         0                   0                  0   
       55.0                         5                   0                  5   

                  deaths  
year   country_id         
1989.0 40.0            0  
       41.0           48  
       41.0            0  
       51.0            1  
       55.0            0

## Merge with GDP dataset

In [106]:
with open(os.path.join("pickle", 'gdp.pickle'), 'rb') as data_source:
    gdp_df = pickle.load(data_source)

In [107]:
gdp_df.head(5)

gdp
year country_id              
1989 2.0         22599.992143
     20.0        20700.799311
     31.0        14213.543372
     40.0         2577.207019
     41.0          393.347528

In [108]:
try: # Set index if not already done
    gdp_df = gdp_df.set_index(["year", "country_id"])
except:
    pass

In [109]:
gdp_df.index.duplicated().sum()

0

In [110]:
human_cost_gdp_df = pd.merge(country_human_cost_df, gdp_df, how='outer',
                             left_index=True, right_index=True, sort=True)

human_cost_gdp_df.fillna(value=0, inplace=True)
    
human_cost_gdp_df.head(5)

number_of_events                         events_id  \
year country_id                                                       
1989 2.0                      0.0                                 0   
     20.0                     0.0                                 0   
     31.0                     0.0                                 0   
     40.0                     0.0                                 0   
     41.0                     4.0  [186087, 186122, 186123, 186124]   

                 displacement_extern  displacement_intern  total_displacement  \
year country_id                                                                 
1989 2.0                         0.0                  0.0                 0.0   
     20.0                        0.0                  0.0                 0.0   
     31.0                        0.0                  0.0                 0.0   
     40.0                        0.0                  0.0                 0.0   
     41.0                        0.0                  0.0                 0.0   

                 deaths  hdi           gdp  
year country_id                             
1989 2.0            0.0  0.0  22599.992143  
     20.0           0.0  0.0  20700.799311  
     31.0           0.0  0.0  14213.543372  
     40.0           0.0  0.0   2577.207019  
     41.0          48.0  0.0    393.347528

In [111]:
country_human_cost_df = human_cost_gdp_df

## Merge with the HDI dataset

In [30]:
with open(os.path.join("pickle", 'hdi.pickle'), 'rb') as data_source:
    hdi_df = pickle.load(data_source)

In [32]:
hdi_df.head(2)

,year,hdi,country_id
0,1990,0.295,700
1,1991,0.300,700


In [33]:
hdi_df = hdi_df.set_index(["year", "country_id"])
print(len(hdi_df))

4267


In [36]:
hdi_duplicated_index = hdi_df.index.duplicated()
hdi_df = hdi_df.loc[~hdi_duplicated_index]
print(len(hdi_df))
hdi_df.head(5)

4162


,,hdi
year,country_id,
1990,700,0.295
1991,700,0.300
1992,700,0.309
1993,700,0.305
1994,700,0.300


In [39]:
human_cost_hdi_df = pd.merge(country_human_cost_df, hdi_df, how='outer',
                             left_index=True, right_index=True, sort=True)

In [42]:
human_cost_hdi_df.fillna(value=0, inplace=True)
    
human_cost_hdi_df.head(5)

number_of_events  \
year country_id                     
1989 41                         4   
     51                         1   
     70                         2   
     90                        78   
     92                        58   

                                                         events_id  \
year country_id                                                      
1989 41                           [186087, 186122, 186123, 186124]   
     51                                                   [107752]   
     70                                           [182047, 182048]   
     90          [193728, 193732, 193764, 193765, 193766, 19376...   
     92          [120534, 120536, 120537, 120538, 120539, 12054...   

                 displacement_extern  displacement_intern  total_displacement  \
year country_id                                                                 
1989 41                            0                    0                   0   
     51                            0                    0                   0   
     70                            0                    0                   0   
     90                            0                    0                   0   
     92                            0                    0                   0   

                 deaths  hdi  
year country_id               
1989 41              48  0.0  
     51               1  0.0  
     70               3  0.0  
     90             491  0.0  
     92            4924  0.0

In [104]:
country_human_cost_df = human_cost_hdi_df

## Add country codes to the dataframe

We add the country codes to be able to display data on map using geopandas and other libraries

In [60]:
country_codes_df = pd.read_csv(os.path.join("data", "countrycode.csv"))
country_codes_df.head(5)

,Unnamed: 0,ar5,continent,cowc,cown,eu28,eurocontrol_pru,eurocontrol_statfor,fao,fips105,...,country.name.es,country.name.fr,country.name.ru,country.name.zh,eurostat,wb_api2c,wb_api3c,p4_scode,p4_ccode,wvs
0,1,ASIA,Asia,AFG,700.0,NaN,Asia,Asia/Pacific,2.0,AF,...,Afganistán,Afghanistan,Афганистан,阿富汗,AF,AF,AFG,AFG,700.0,4.0
1,2,OECD1990,Europe,NaN,NaN,NaN,Eurocontrol,ESRA North-West,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,EIT,Europe,ALB,339.0,NaN,Eurocontrol,ESRA East,3.0,AL,...,Albania,Albanie,Албания,阿尔巴尼亚,AL,AL,ALB,ALB,339.0,8.0
3,4,MAF,Africa,ALG,615.0,NaN,Africa,North-Africa,4.0,AG,...,Argelia,Algérie,Алжир,阿尔及利亚,DZ,DZ,DZA,ALG,615.0,12.0
4,5,ASIA,Oceania,NaN,NaN,NaN,Asia,Asia/Pacific,NaN,AQ,...,NaN,NaN,NaN,NaN,AS,AS,ASM,NaN,NaN,16.0


In [61]:
def extract_iso3_from_gnow(gnwo):
    """Get the iso3 country code with the gnwo value, return None if no match was found"""
    try:
        iso3 = country_codes_df.loc[country_codes_df.cown == gnwo]["wb_api3c"].values[0]
    except IndexError as e:
        # 99 = Great Columbia
        # 711 = Tibet
        # 972 = Tonga
        # 973 = Tuvalu
        if not (gnwo == 99 or gnwo == 711 or gnwo == 971 or gnwo == 972 or gnwo == 973 or pd.isnull(gnwo) or gnwo == 340):
            print(e, "with gnwo: {}".format(gnwo))
        return None
    return iso3
print(extract_iso3_from_gnow(2))

USA


In [62]:
def extract_gnwo_countries_to_df():
    """Extract the countries from the gnwo and their id from the gnwo files"""
    countries_list = []
    # First gnwo file, contains id, code, name for all gnow numbers
    with open(os.path.join("data", "gnwo.txt"), "r") as gnow:
        for line in gnow:
            split_line = re.split(r'\t+', line)
            countries_list.append(split_line[0:3])
    countries_df = pd.DataFrame(countries_list, columns=["id", "code", "name"])
    countries_df["id"] = pd.to_numeric(countries_df["id"])
    countries_df = countries_df.drop_duplicates()
    return countries_df.set_index("id")
countries_list = extract_gnwo_countries_to_df()

countries_list.head(5)

,code,name
id,,
2,USA,United States of America
20,CAN,Canada
31,BHM,Bahamas
40,CUB,Cuba
41,HAI,Haiti


In [63]:
# Add code column to the country_human_cost_df
countries_codes = []
for row in country_human_cost_df.itertuples():
    countries_codes.append([row.Index[0], row.Index[1], extract_iso3_from_gnow(row.Index[1])])
countries_codes = pd.DataFrame(countries_codes, columns=["year", "country_id","country_code"])\
                    .set_index(["year", "country_id"])

In [64]:
if not "country_code" in country_human_cost_df.columns:
    country_human_cost_df = pd.concat([country_human_cost_df, countries_codes], axis=1, join_axes=[country_human_cost_df.index])

country_human_cost_df = country_human_cost_df[~country_human_cost_df.country_code.isnull()]
country_human_cost_df.head(5)

number_of_events  \
year country_id                     
1989 41                         4   
     51                         1   
     70                         2   
     90                        78   
     92                        58   

                                                         events_id  \
year country_id                                                      
1989 41                           [186087, 186122, 186123, 186124]   
     51                                                   [107752]   
     70                                           [182047, 182048]   
     90          [193728, 193732, 193764, 193765, 193766, 19376...   
     92          [120534, 120536, 120537, 120538, 120539, 12054...   

                 displacement_extern  displacement_intern  total_displacement  \
year country_id                                                                 
1989 41                            0                    0                   0   
     51                            0                    0                   0   
     70                            0                    0                   0   
     90                            0                    0                   0   
     92                            0                    0                   0   

                 deaths  hdi country_code  
year country_id                            
1989 41              48  0.0          HTI  
     51               1  0.0          JAM  
     70               3  0.0          MEX  
     90             491  0.0          GTM  
     92            4924  0.0          SLV

In [65]:
country_human_cost_df.country_code.isnull().any()

False

In [67]:
# Compare the obtained country code with the one in the geopanda world dataset
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world.head(5)

,pop_est,continent,name,iso_a3,gdp_md_est,geometry
0,28400000.0,Asia,Afghanistan,AFG,22270.0,"POLYGON ((61.21081709172574 35.65007233330923,..."
1,12799293.0,Africa,Angola,AGO,110300.0,(POLYGON ((16.32652835456705 -5.87747039146621...
2,3639453.0,Europe,Albania,ALB,21810.0,"POLYGON ((20.59024743010491 41.85540416113361,..."
3,4798491.0,Asia,United Arab Emirates,ARE,184300.0,"POLYGON ((51.57951867046327 24.24549713795111,..."
4,40913584.0,South America,Argentina,ARG,573900.0,(POLYGON ((-65.50000000000003 -55.199999999999...


In [69]:
# Drop country for which we don't have a corresponding country code
country_code_to_drop = []
for country_code in country_human_cost_df.country_code.unique():
    if not world.iso_a3.str.contains(country_code).any():
        country_code_to_drop.append(country_code)

# Remove unrecognized countries from the dataframe
country_human_cost_df = country_human_cost_df[~country_human_cost_df.country_code.isin(country_code_to_drop)]
country_human_cost_df.head(5)

number_of_events  \
year country_id                     
1989 41                         4   
     51                         1   
     70                         2   
     90                        78   
     92                        58   

                                                         events_id  \
year country_id                                                      
1989 41                           [186087, 186122, 186123, 186124]   
     51                                                   [107752]   
     70                                           [182047, 182048]   
     90          [193728, 193732, 193764, 193765, 193766, 19376...   
     92          [120534, 120536, 120537, 120538, 120539, 12054...   

                 displacement_extern  displacement_intern  total_displacement  \
year country_id                                                                 
1989 41                            0                    0                   0   
     51                            0                    0                   0   
     70                            0                    0                   0   
     90                            0                    0                   0   
     92                            0                    0                   0   

                 deaths  hdi country_code  
year country_id                            
1989 41              48  0.0          HTI  
     51               1  0.0          JAM  
     70               3  0.0          MEX  
     90             491  0.0          GTM  
     92            4924  0.0          SLV

## Pickle the result

In [70]:
with open(os.path.join("pickle", 'country_human_cost.pickle'), 'wb') as out:
    pickle.dump(country_human_cost_df, out)

In [71]:
with open(os.path.join("pickle", 'country_human_cost.pickle'), 'rb') as data_source:
    country_human_cost_df = pickle.load(data_source)